In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

response = client.invoke([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "こんにちは！私はジョンといいます。"}
])
print(response)

content='こんにちは、ジョンさん！私はアシスタントです。何かお手伝いできることはありますか？' additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019b647b-df2c-7450-b7d6-1a4be3a26d44-0' usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}}


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1",
    streaming=True
)

response = client.invoke([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "こんにちは！私はジョンといいます。"},

])
print(response)


content='こんにちは、ジョンさん！お会いできて嬉しいです。何かお手伝いできることはありますか？\n' additional_kwargs={} response_metadata={'safety_ratings': [], 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'model_provider': 'google_genai'} id='lc_run--019b647b-e4dc-7ae2-9002-06aaef1c0aed' usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}}


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1",
    response_mime_type="application/json"
)

response = client.invoke([
    {"role": "system", "content": "人物一覧を次のJSON形式で出力してください。[{\"name\": \"\", \"age\": \"\", \"city\": \"\"}]"},
    {"role": "user", "content": "以下の人物について教えてください。佐藤太郎50歳、東京都出身です。普段は会社に勤めてます。鈴木花子30歳、大阪府出身です。フリーランスで働いています。"},
])

print(response.content)

```json
[
  {"name": "佐藤太郎", "age": "50", "city": "東京都"},
  {"name": "鈴木花子", "age": "30", "city": "大阪府"}
]
```


In [4]:
# ====================================
# Function Calling 入門
# ====================================
# AIが「どの関数を呼ぶべきか」を判断し、引数を生成する機能

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool

# 1. ツール（関数）を定義
@tool
def get_weather(city: str) -> str:
    """指定した都市の天気を取得する"""
    # 実際はAPIを呼ぶが、ここではダミーデータ
    weather_data = {
        "東京": "晴れ 25℃",
        "大阪": "曇り 22℃",
        "名古屋": "雨 18℃"
    }
    return weather_data.get(city, f"{city}の天気情報はありません")

@tool
def calculate(expression: str) -> str:
    """数式を計算する"""
    try:
        result = eval(expression)
        return f"計算結果: {result}"
    except:
        return "計算できませんでした"

# 2. LLMにツールをバインド
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

llm_with_tools = llm.bind_tools([get_weather, calculate])

# 3. 質問してみる
response = llm_with_tools.invoke("大分県の天気は？")
#今まではresponse = llm.invoke{[
#    {"role": "system", "content": "人物一覧を次のJSON形式で出力してください。[{\"name\": \"\", \"age\": \"\", \"city\": \"\"}]"},
#    {"role": "user", "content": "以下の人物について教えてください。佐藤太郎50歳、東京都出身です。普段は会社に勤めてます。鈴木花子30歳、大阪府出身です。フリーランスで働いています。"},
#]}


print("=== AIの判断 ===")
print(f"呼び出すツール: {response.tool_calls}")
print()

# 4. ツールを実際に実行
if response.tool_calls:
    tool_call = response.tool_calls[0]
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]

    print(f"=== ツール実行 ===")
    print(f"ツール名: {tool_name}")
    print(f"引数: {tool_args}")

    # ツールを実行
    if tool_name == "get_weather":
        result = get_weather.invoke(tool_args)
    elif tool_name == "calculate":
        result = calculate.invoke(tool_args)

    print(f"結果: {result}")

=== AIの判断 ===
呼び出すツール: [{'name': 'get_weather', 'args': {'city': '大分県'}, 'id': '3a28741e-8245-434a-beda-fdc8853cffcc', 'type': 'tool_call'}]

=== ツール実行 ===
ツール名: get_weather
引数: {'city': '大分県'}
結果: 大分県の天気情報はありません


In [5]:
import json

def get_current_weather(location,unint="fahrenheit"):
    if "tokyo" in location.lower():
        return {
            "location": "Tokyo",
            "temperature": "22",
            "unit": unint,
            "forecast": "sunny"
        }
    elif "osaka" in location.lower():
        return {
            "location": "Osaka",
            "temperature": "25",
            "unit": unint,
            "forecast": "cloudy"
        }
    else:
        return {
            "location": location,
            "temperature": "unknown",
            "unit": unint,
            "forecast": "unknown"
        }

In [6]:
# ====================================
# Function Calling × SQL
# ====================================
# 自然言語からSQLを生成・実行する

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
import sqlite3

# 1. インメモリDBを作成してサンプルデータを投入
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER,
        city TEXT,
        department TEXT
    )
""")

# サンプルデータ
sample_users = [
    ("山田太郎", 30, "東京", "営業部"),
    ("鈴木花子", 25, "大阪", "開発部"),
    ("佐藤次郎", 28, "名古屋", "営業部"),
    ("田中美咲", 35, "東京", "人事部"),
    ("高橋健一", 42, "福岡", "開発部"),
    ("伊藤さくら", 23, "東京", "開発部"),
]

cursor.executemany("INSERT INTO users (name, age, city, department) VALUES (?, ?, ?, ?)", sample_users)
conn.commit()

print("=== データベース作成完了 ===")
print("テーブル: users (id, name, age, city, department)")
print(f"データ件数: {len(sample_users)}件")
print()

# 全データ表示
cursor.execute("SELECT * FROM users")
for row in cursor.fetchall():
    print(row)

=== データベース作成完了 ===
テーブル: users (id, name, age, city, department)
データ件数: 6件

(1, '山田太郎', 30, '東京', '営業部')
(2, '鈴木花子', 25, '大阪', '開発部')
(3, '佐藤次郎', 28, '名古屋', '営業部')
(4, '田中美咲', 35, '東京', '人事部')
(5, '高橋健一', 42, '福岡', '開発部')
(6, '伊藤さくら', 23, '東京', '開発部')


In [7]:
# 2. SQLを実行するツールを定義
@tool
def execute_sql(query: str) -> str:
    """
    SQLクエリを実行してデータベースから情報を取得する。

    テーブル: users
    カラム: id, name(名前), age(年齢), city(都市), department(部署)

    例:
    - SELECT * FROM users WHERE city = '東京'
    - SELECT * FROM users WHERE age >= 30
    - SELECT department, COUNT(*) FROM users GROUP BY department
    """
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        if results:
            columns = [desc[0] for desc in cursor.description]
            output = f"カラム: {columns}\n"
            for row in results:
                output += f"{row}\n"
            return output
        return "該当するデータがありませんでした"
    except Exception as e:
        return f"エラー: {e}"

# 3. LLMにツールをバインド
llm_sql = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

llm_with_sql = llm_sql.bind_tools([execute_sql])

print("=== SQLツール設定完了 ===")

=== SQLツール設定完了 ===


In [8]:
# 4. 自然言語で質問してみる
def ask_database(question: str):
    """自然言語でDBに質問する"""
    print(f"質問: {question}")
    print("-" * 40)

    response = llm_with_sql.invoke(question)

    if response.tool_calls:
        tool_call = response.tool_calls[0]
        sql_query = tool_call["args"]["query"]

        print(f"生成されたSQL: {sql_query}")
        print("-" * 40)

        result = execute_sql.invoke({"query": sql_query})
        print(f"結果:\n{result}")
    else:
        print(f"AIの回答: {response.content}")

    print("=" * 50)
    print()

# いくつかの質問を試す
ask_database("東京に住んでいる人を全員教えて")

質問: 東京に住んでいる人を全員教えて
----------------------------------------
生成されたSQL: SELECT * FROM users WHERE city = '東京'
----------------------------------------
結果:
カラム: ['id', 'name', 'age', 'city', 'department']
(1, '山田太郎', 30, '東京', '営業部')
(4, '田中美咲', 35, '東京', '人事部')
(6, '伊藤さくら', 23, '東京', '開発部')




In [9]:
# 他の質問も試してみよう
ask_database("30歳以上の社員は誰？")
ask_database("部署ごとの人数を教えて")
ask_database("開発部で一番若い人は？")

質問: 30歳以上の社員は誰？
----------------------------------------
AIの回答: 次のSQLクエリを実行して、30歳以上の社員の名前を取得できます。
```sql
SELECT name FROM users WHERE age >= 30
```

質問: 部署ごとの人数を教えて
----------------------------------------
生成されたSQL: SELECT department, COUNT(*) FROM users GROUP BY department
----------------------------------------
結果:
カラム: ['department', 'COUNT(*)']
('人事部', 1)
('営業部', 2)
('開発部', 3)


質問: 開発部で一番若い人は？
----------------------------------------
生成されたSQL: SELECT * FROM users WHERE department = '開発部' ORDER BY age ASC LIMIT 1
----------------------------------------
結果:
カラム: ['id', 'name', 'age', 'city', 'department']
(6, '伊藤さくら', 23, '東京', '開発部')




In [10]:
prompt ='''\
以下の料理のレシピを教えてください。

料理名: """
{dish}
"""
'''

def get_recipe(dish_name: str) -> str:
    response = llm.invoke([
        {"role": "user", "content": prompt.format(dish=dish_name)}
    ])
    return response.content

get_recipe("カレーライス")
#prompt.format(dish="田中")

'はい、承知いたしました。カレーライスのレシピを以下に示します。\n\n**材料（4人分）**\n\n*   **肉:** 豚肉、牛肉、鶏肉などお好みで 300g (一口大に切る)\n*   **玉ねぎ:** 大 2個 (薄切り)\n*   **にんじん:** 中 1本 (乱切り)\n*   **じゃがいも:** 中 3個 (一口大に切る。メークインなど煮崩れしにくいものがおすすめ)\n*   **カレールー:** 4人分 (市販のカレールーを使用)\n*   **サラダ油:** 大さじ2\n*   **水:** カレールーの箱に記載されている分量\n*   **ご飯:** 適量 (炊きたて)\n*   **（あれば）ローリエ:** 1枚\n*   **（あれば）ケチャップ:** 大さじ1\n*   **（あれば）ウスターソース:** 大さじ1\n\n**作り方**\n\n1.  **下準備:**\n    *   野菜を洗い、切る。\n    *   肉に軽く塩コショウ（分量外）を振っておく。\n\n2.  **炒める:**\n    *   鍋にサラダ油を熱し、玉ねぎを炒める。\n        *   焦げ付かないように弱火〜中火で、あめ色になるまでじっくり炒めると甘みが増して美味しくなる。\n    *   玉ねぎがしんなりしたら、肉を加えて炒める。肉の色が変わったら、にんじん、じゃがいもを加えて炒める。\n    *   全体に油が回ったら、水を加えて沸騰させる。\n\n3.  **煮込む:**\n    *   アクを取り、ローリエを加える（あれば）。\n    *   野菜が柔らかくなるまで、弱火で15〜20分ほど煮込む。\n    *   じゃがいもが柔らかくなっているか確認する。竹串などを刺して確認しましょう。\n\n4.  **ルーを加える:**\n    *   一旦火を止め、カレールーを割り入れ、完全に溶かす。\n        *   ルーが溶けにくい場合は、少しずつ加えながら混ぜると良い。\n    *   再び弱火にかけ、とろみがつくまで混ぜながら5〜10分ほど煮込む。\n    *   焦げ付かないように、時々混ぜる。\n    *   味見をして、必要であればケチャップ、ウスターソースを加えて味を調える。\n        * ケチ

In [11]:
# ====================================
# LangSmith トレース例（langchain-google-vertexai使用）
# ====================================

from dotenv import load_dotenv
load_dotenv()  # LangSmithの設定を読み込む

from langchain_google_genai import ChatGoogleGenerativeAI

# Vertex AI経由でGeminiを使用
llm_vertex = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# 基本的な使い方
response = llm_vertex.invoke("こんにちは！今日の気分はどう？")
print("=== 基本的な呼び出し ===")
print(response.content)
print()

# メッセージ形式
response2 = llm_vertex.invoke([
    {"role": "system", "content": "あなたは関西弁で話すアシスタントです。"},
    {"role": "user", "content": "自己紹介してください"}
])
print("=== システムプロンプト付き ===")
print(response2.content)

=== 基本的な呼び出し ===
こんにちは！私はおかげさまで今日も快調です。何かお手伝いできることはありますか？😊

=== システムプロンプト付き ===
へい、毎度！わいは関西弁でおしゃべりするアシスタントやで。なんでも聞いてや。おもろい話も、真面目な相談も、どんとこいや！😊


In [12]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["""以下の料理のレシピを考えてください。料理名 {dish}"""
])

prompt_value = prompt.invoke({"dish": "カレー"})


In [13]:
# ChatPromptValue の中身を確認する方法
print("=== 文字列として表示 ===")
print(prompt_value.to_string())

print("\n=== メッセージとして表示 ===")
print(prompt_value.to_messages())

=== 文字列として表示 ===
Human: 以下の料理のレシピを考えてください。料理名 カレー

=== メッセージとして表示 ===
[HumanMessage(content='以下の料理のレシピを考えてください。料理名 カレー', additional_kwargs={}, response_metadata={})]


In [14]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です。"),
    ("human", "以下の料理のレシピを考えてください。料理名 {dish}")
])

prompt_value = prompt.invoke({"dish": "オムライス"})


In [15]:
print("=== 文字列として表示 ===")
print(prompt_value.to_string())

=== 文字列として表示 ===
System: あなたは料理の専門家です。
Human: 以下の料理のレシピを考えてください。料理名 オムライス


In [16]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain_core.tools import tool

prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です。"),
    MessagesPlaceholder(variable_name="history", optional=True),
    ("human", "以下の料理のレシピを考えてください。料理名 {dish}")
])

prompt_value = prompt.invoke({"dish": "オムライス"})
print("=== 文字列として表示 ===")
print(prompt_value.to_string())

=== 文字列として表示 ===
System: あなたは料理の専門家です。
Human: 以下の料理のレシピを考えてください。料理名 オムライス


In [17]:
prompt_value = prompt.invoke({
    "dish": "カレー",
    "history": [
        ("human", "カレーのレシピを教えて"),
        ("ai", "カレーのレシピは玉ねぎを炒めて..."),
        ("human", "辛くするには？"),
        ("ai", "スパイスを増やすと辛くなります"),
        ("human", "激辛が好きなんだよ"),
    ]
})

In [18]:
print("=== 文字列として表示 ===")
print(llm.invoke(prompt_value).content)

=== 文字列として表示 ===
はい、承知いたしました。激辛がお好きとのことですので、以下の激辛カレーのレシピをご提案します。

**料理名：** 悶絶必至！激辛デビルカレー

**材料（4人分）：**

*   **肉：**
    *   豚バラブロック または 牛すじ：300g (お好みで)
*   **野菜：**
    *   玉ねぎ：大きめ2個（薄切り）
    *   人参：1本（乱切り）
    *   じゃがいも：2個（乱切り）
    *   ニンニク：2かけ（みじん切り）
    *   生姜：1かけ（みじん切り）
    *   ピーマン：2個（種を取り乱切り）
*   **カレールー：**
    *   市販のカレールー（辛口）：1箱
*   **激辛スパイス（お好みで量を調整）：**
    *   カイエンペッパー：小さじ2〜 大さじ1
    *   ハバネロパウダー：小さじ1/2〜 小さじ1
    *   チリパウダー：大さじ1〜 大さじ2
    *   一味唐辛子：小さじ1〜 大さじ2
    *   **おすすめ:** ブートジョロキアパウダー：少量（耳かき1〜2杯程度から試す。非常に辛いので注意！）
*   **その他：**
    *   サラダ油：大さじ2
    *   水：700ml
    *   ウスターソース：大さじ1
    *   ケチャップ：大さじ1
    *   醤油：小さじ1
    *   ローリエ：1枚
    *   ガラムマサラ：小さじ1/2（仕上げに）
    *   **隠し味におすすめ:** チョコレート：1かけ(5g程度)
*   **付け合わせ（お好みで）：**
    *   ご飯
    *   福神漬け、らっきょう

**作り方：**

1.  **下準備：**
    *   豚バラブロックを使う場合は、2〜3cm角に切る。牛すじを使う場合は、下茹でしてアクを取り、食べやすい大きさに切る。
    *   玉ねぎは薄切り、人参、じゃがいもは乱切り、ニンニクと生姜はみじん切りにする。
2.  **炒め：**
    *   鍋にサラダ油を熱し、ニンニクと生姜を炒める。香りが立ったら玉ねぎを加えて、あめ色になるまでじっくり炒める。
    *   豚肉または牛すじを加え、

In [19]:
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# 準備
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です"),
    ("human", "{dish}のレシピを教えて")
])

# 実行（パイプでつなぐ）
chain = prompt | llm
result = chain.invoke({"dish": "カレー"})
print(result.content)

はい、承知いたしました。本格的なものから簡単なものまで、色々なカレーのレシピがありますが、今回は基本の**ポークカレー**のレシピをご紹介します。

**材料（4～5人分）**

*   豚肩ロース塊：300g
*   玉ねぎ：中2個
*   にんじん：1本
*   じゃがいも：中2個
*   カレールー：1箱（8皿分）
*   サラダ油：大さじ2
*   水：700ml
*   あれば、ローリエ：1枚

**作り方**

1.  **下準備:**
    *   豚肉は2～3cm角に切る。
    *   玉ねぎは薄切りにする。
    *   にんじん、じゃがいもは皮をむいて乱切りにする。
2.  **炒める:**
    *   鍋にサラダ油を熱し、豚肉を炒める。表面に焼き色がついたら、玉ねぎを加えて炒める。
    *   玉ねぎがしんなりしたら、にんじん、じゃがいもを加えて炒め合わせる。
3.  **煮込む:**
    *   水を加えて沸騰したらアクを取り、ローリエを加えて弱火で20分ほど煮込む。
    *   野菜が柔らかくなったら、火を止めてカレールーを割り入れ、溶けるまでよく混ぜる。
4.  **仕上げ:**
    *   再び弱火でとろみがつくまで煮込む。焦げ付かないように時々混ぜる。
5.  **盛り付け:**
    *   ご飯と一緒に皿に盛り付ければ完成！

**美味しく作るポイント**

*   **豚肉:** 豚バラ肉や鶏もも肉でも美味しく作れます。
*   **玉ねぎ:** じっくり炒めることで甘みが増し、カレーが美味しくなります。
*   **煮込み時間:** 煮込むほど具材が柔らかくなり、味が染み込みます。
*   **隠し味:** チョコレート、コーヒー、ソースなどを加えると、より深みのある味わいになります。
*   **辛さ調整:** カレールーの種類を変える、チリパウダーやラー油を加えるなどで辛さを調整できます。

**アレンジ**

*   **シーフードカレー:** 豚肉をエビやイカなどのシーフードに変えて、ココナッツミルクを加えるとエスニックな味わいになります。
*   **キーマカレー:** 豚肉をひき肉に変えて、トマトやスパイスを加えて炒め煮にすると、キーマカレーになります。
*    **野菜カレー:**

In [20]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser


class Recipe(BaseModel):
  ingredients: list[str] = Field(description="料理の材料のリスト")
  steps: list[str] = Field(description="料理の手順のリスト")


output_parser = PydanticOutputParser(pydantic_object=Recipe)

format_instructions = output_parser.get_format_instructions()
print("=== 文字列として表示 ===")
print(format_instructions)


=== 文字列として表示 ===
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "料理の材料のリスト", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "料理の手順のリスト", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# 型定義
class Recipe(BaseModel):
    ingredients: list[str] = Field(description="料理の材料のリスト")
    steps: list[str] = Field(description="料理の手順のリスト")

# パーサー
output_parser = PydanticOutputParser(pydantic_object=Recipe)

# LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# プロンプト
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です。"),
    ("human", "以下の料理のレシピを考えてください。料理名 {dish}\n\n{format_instructions}")
])

# チェーン（output_parserを追加）
chain = prompt | llm | output_parser

# 実行
result = chain.invoke({
    "dish": "オムライス",
    "format_instructions": output_parser.get_format_instructions()
})

print(result)
# → Recipe(ingredients=[...], steps=[...])


ingredients=['鶏もも肉 100g', '玉ねぎ 1/4個', 'マッシュルーム 3個', 'ご飯 200g', '卵 3個', '牛乳 大さじ1', 'バター 10g', 'ケチャップ 大さじ3', '塩 少々', 'こしょう 少々', 'サラダ油 大さじ1', 'ケチャップ (仕上げ用) 適量'] steps=['鶏肉、玉ねぎ、マッシュルームを細かく切る。', 'フライパンにサラダ油を熱し、鶏肉を炒める。色が変わったら玉ねぎ、マッシュルームを加えて炒める。', 'ご飯を加えて炒め、ケチャップ、塩、こしょうで味付けし、ケチャップライスを作る。', 'ボウルに卵を割りほぐし、牛乳、塩、こしょうを加えて混ぜる。', 'フライパンにバターを熱し、卵液を流し込む。軽く混ぜながら半熟状にする。', 'ケチャップライスを卵で包み、皿に盛り付ける。', '仕上げにケチャップをかける。']


In [26]:
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="料理の材料のリスト")
    steps: list[str] = Field(description="料理の手順のリスト")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# これだけでOK！
structured_llm = llm.with_structured_output(Recipe)
result = structured_llm.invoke("オムライスのレシピを教えて")

print(result.ingredients)
print(result.steps)


['ご飯: 2膳', '鶏もも肉: 100g', '玉ねぎ: 1/4個', '人参: 1/4本', 'マッシュルーム: 3個', '卵: 3個', '牛乳: 大さじ1', 'バター: 10g', 'ケチャップ: 大さじ3', 'コンソメ: 小さじ1/2', '塩: 少々', 'こしょう: 少々', 'サラダ油: 適量', 'ケチャップ (仕上げ用): 適量']
['鶏もも肉、玉ねぎ、人参、マッシュルームをそれぞれ1cm角に切る。', 'フライパンにサラダ油を熱し、鶏もも肉を炒める。色が変わったら玉ねぎ、人参、マッシュルームを加えて炒める。', '野菜がしんなりしたらご飯を加え、ケチャップ、コンソメ、塩、こしょうで味付けし、ケチャップライスを作る。お皿に移しておく。', 'ボウルに卵、牛乳、塩、こしょうを混ぜ合わせる。', 'フライパンにバターを熱し、卵液を流し込む。半熟状になったら火を止め、ケチャップライスを中央に置く。', '卵でケチャップライスを包み込むように形を整え、お皿に盛り付ける。', '仕上げにケチャップをかけたら完成。']


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# 1つ目: 料理名を考える
prompt1 = ChatPromptTemplate.from_template("{ingredient}を使った料理名を1つだけ答えて")

# 2つ目: その料理のレシピを考える
prompt2 = ChatPromptTemplate.from_template("{dish}のレシピを教えて")

# チェーンをつなげる
chain = (
    prompt1
    | llm
    | StrOutputParser()  # ← ここで文字列に変換
    | (lambda dish: {"dish": dish})  # ← 次のプロンプトに渡す
    | prompt2
    | llm
    | StrOutputParser()
)

result = chain.invoke({"ingredient": "卵"})
print(result)
# 1. "卵"を使った料理名 → "オムライス"
# 2. "オムライス"のレシピ → "材料は..."


はい、美味しい卵焼きのレシピをご紹介します！基本の卵焼きから、少しアレンジを加えたものまでありますので、お好みのものを選んでみてください。

**基本の卵焼き**

材料：

*   卵：3個
*   だし汁：大さじ3
*   砂糖：大さじ1
*   醤油：小さじ1/2
*   サラダ油：適量

作り方：

1.  ボウルに卵を割りほぐし、だし汁、砂糖、醤油を加えてよく混ぜ合わせます。
2.  卵焼き器を中火で熱し、サラダ油を薄くひきます。
3.  卵液を少量流し込み、表面が固まってきたら奥に巻き込みます。
4.  空いたところに再び油をひき、卵液を流し込んで同様に巻き込みます。
5.  これを繰り返し、卵液がなくなるまで焼き上げます。
6.  巻き終わりを下にして形を整え、粗熱を取ってから切り分けます。

**アレンジ卵焼き**

*   **ネギ入り卵焼き：** 刻んだネギを卵液に混ぜて焼きます。
*   **チーズ入り卵焼き：** 卵を巻き込む際に、細かく切ったチーズを加えます。
*   **明太マヨ卵焼き：** 卵液に明太子とマヨネーズを混ぜて焼きます。
*   **だし巻き卵：** だし汁を多め（卵1個あたり大さじ2〜3）にして、砂糖と醤油を控えめにすると、よりしっとりとしただし巻き卵になります。

**美味しく作るコツ：**

*   卵液は混ぜすぎないように、泡立て器ではなく菜箸で切るように混ぜると、ふっくらと仕上がります。
*   卵焼き器は十分に温めてから油をひくと、卵がくっつきにくくなります。
*   卵焼き器の火加減は、常に中火〜弱火を保つようにしましょう。強すぎると焦げ付きやすく、弱すぎると時間がかかりすぎて硬くなります。
*   卵液を流し込む量は、一度にたくさん入れすぎないように、薄く均一に広げることがポイントです。
*   巻き終わりは軽く押さえるようにすると、形が崩れにくくなります。

**その他：**

*   甘さの調整：砂糖の量はお好みで調整してください。甘めが好きな方は少し多めに、甘さ控えめが好きな方は少なめにしてください。
*   味付けのバリエーション：醤油の代わりに白だしや麺つゆを使うと、また違った風味になります。
*   具材の追加：上記以外にも、桜エビや三つ葉、海苔など、お好みの具材を加えてアレンジできます。

ぜひ

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# 型定義
class Recipe(BaseModel):
    date: list[str] = Field(description="日付")
    menue: list[str] = Field(description="料理名")
    ingredients: list[str] = Field(description="料理の材料のリスト")
    steps: list[str] = Field(description="料理の手順のリスト")

# 1つ目: 料理名を考える
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です。料理名だけを1つ答えてください。"),
    ("human", "{ingredient}を使った料理は？")
])

# 2つ目: レシピを構造化して取得
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です。"),
    ("human", "{dish}のレシピを教えて")
])

structured_llm = llm.with_structured_output(Recipe)

# チェーン
chain = (
    prompt1
    | llm
    | StrOutputParser()
    | (lambda dish: {"dish": dish})  # ← 辞書に変換
    | prompt2
    | structured_llm
)

result = chain.invoke({"ingredient": "卵"})
print(result.date)
print(result.menue)
print(result.ingredients)
print(result.steps)


['2024-01-02']
['オムライス']
['卵 (3個)', '鶏もも肉 (80g)', '玉ねぎ (1/4個)', 'ごはん (茶碗2杯分)', 'ケチャップ (大さじ3)', 'バター (10g)', 'サラダ油 (適量)', '塩 (少々)', 'こしょう (少々)', '牛乳 (大さじ1)', 'パセリ (みじん切り、お好みで)']
['鶏肉と玉ねぎを細かく切る。', 'フライパンにサラダ油を熱し、鶏肉と玉ねぎを炒める。鶏肉に火が通ったら、ごはんを加えて炒め、ケチャップ、塩、こしょうで味を調える。ケチャップライスを皿に盛る。', 'ボウルに卵を割り入れ、牛乳、塩、こしょうを加えてよく混ぜる。', 'フライパンにバターを熱し、卵液を流し込む。半熟状になったら、ケチャップライスの上に滑らせるように乗せる。', 'お好みでケチャップやパセリをかけて完成。']


In [ ]:
# ====================================
# RAG（Retrieval-Augmented Generation）入門
# ====================================
# 自分のデータを検索して、LLMに回答させる仕組み

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM準備
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# ====================================
# 簡易版: ダミーのRetrieverを作成
# ====================================
# 本番では Vertex AI Search や ChromaDB を使う
# ここでは仕組みを理解するために、簡単な検索関数を作成

# サンプルデータ（本番ではVector Storeに保存）
documents = {
    "りんご": "りんごは青森県が生産量日本一です。糖度が高く、ふじ、つがる、王林などの品種があります。旬は秋から冬です。",
    "みかん": "みかんは和歌山県や愛媛県が有名な産地です。ビタミンCが豊富で、冬の定番果物です。",
    "バナナ": "バナナはフィリピンやエクアドルからの輸入が多いです。栄養価が高く、手軽に食べられる果物です。",
    "ぶどう": "ぶどうは山梨県が有名です。巨峰やシャインマスカットなどの品種があります。ワインの原料にもなります。",
}

def simple_retriever(query: str) -> str:
    """簡易検索: キーワードが含まれるドキュメントを返す"""
    results = []
    for keyword, content in documents.items():
        if keyword in query or any(word in query for word in ["果物", "フルーツ", "産地"]):
            results.append(f"【{keyword}について】\n{content}")

    if results:
        return "\n\n".join(results)
    return "該当する情報が見つかりませんでした。"

# ====================================
# RAGチェーンの構築
# ====================================

# プロンプト
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは果物の専門家です。以下の検索結果を元に質問に答えてください。検索結果にない情報は「わかりません」と答えてください。"),
    ("human", "検索結果:\n{context}\n\n質問: {question}")
])

# チェーン
# {"context": ..., "question": ...} の形でpromptに渡す
chain = (
    {
        "context": lambda x: simple_retriever(x),  # 検索実行
        "question": lambda x: x                     # 質問はそのまま（これが主流）
    }
    | prompt
    | llm
    | StrOutputParser()
)

# ====================================
# 実行例
# ====================================
print("=== RAG実行例 ===\n")

# 質問1: データに含まれる情報
question1 = "りんごの産地はどこ？"
print(f"質問: {question1}")
print(f"回答: {chain.invoke(question1)}\n")
print("-" * 50)

# 質問2: データに含まれない情報
question2 = "いちごの産地はどこ？"
print(f"質問: {question2}")
print(f"回答: {chain.invoke(question2)}\n")

=== RAG実行例 ===

質問: りんごの産地はどこ？
回答: りんごは青森県が生産量日本一です。

--------------------------------------------------
質問: いちごの産地はどこ？
回答: わかりません。

